<a href="https://colab.research.google.com/github/Teoroo-CMC/DoE_Course_Material/blob/main/Week_2/workshop_4/Jupyter-notebooks/2-6factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# More analysis of the six-Factor Full Factorial Design

## Introduction
Let us continue where we ended in Task 2. 

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from numpy.random import rand, seed
import seaborn as sns
import scipy.stats as stats
from matplotlib.pyplot import *

seed(10)

## Two-Level Six-Factor Full Factorial Design
First, let's recover the data from Task 2. 

In [ ]:
import itertools

# Create the inputs:
encoded_inputs = list( itertools.product([-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1]) )

# Create the experiment design table:
doe=pd.DataFrame(encoded_inputs)
doe=doe[doe.columns[::-1]]
doe.columns=['x%d'%(i+1) for i in range(6)]
doe

doe['y1'] = [3.4, 9.7, 7.4, 10.6, 6.5, 7.9, 10.3, 9.5, 14.3, 10.5, 7.8, 17.2, 9.4, 12.1, 9.5, 15.8, 8.3, 8.0, 7.9, 10.7, 7.2, 7.2, 7.9, 10.2, 10.3, 9.9, 7.4, 10.5, 9.6, 15.1, 8.7, 12.1, 12.6, 10.5, 11.3, 10.6, 8.1, 12.5, 11.1, 12.9, 14.6, 12.7, 10.8, 17.1, 13.6, 14.6, 13.3, 14.4, 11.0, 12.5, 8.9, 13.1, 7.6, 8.6, 11.8, 12.4, 13.4, 14.6, 14.9, 11.8, 15.6, 12.8, 13.5, 15.8] 
doe['y2'] = [15,5,23, 8,20, 9, 13, 5, 23, 1, 11, 5, 15, 8, 15, 1, 22, 8, 16, 7, 25, 5, 17, 8, 10, 3, 22, 6, 24, 4, 10, 5, 32, 10, 28, 18, 22, 31, 17, 16, 38, 12, 34, 19, 12, 14, 25, 16, 31, 14, 23, 23, 28, 20, 18, 11, 39, 30, 31, 6, 33, 23, 31, 11]
doe['y3'] = [36, 35, 37, 34, 30, 32, 28, 38, 40, 32, 32, 28, 34, 26,  30, 28, 40, 30, 35, 35, 32, 35, 36, 32, 20, 35, 35, 28, 27, 36, 36, 35, 32, 34, 30, 24, 30, 20, 32, 25, 20, 20, 22, 35, 26, 15, 19, 24, 22, 23, 22, 18, 20, 20, 20, 36, 20, 11, 20, 35, 16, 32, 20, 20]
print(doe[['y1','y2','y3']])

# Defining Variables and Variable Labels

labels = {}
labels[1] = ['x1','x2','x3','x4','x5','x6']
for i in [2,3,4,5,6]:
    labels[i] = list(itertools.combinations(labels[1], i))

obs_list = ['y1','y2','y3']

for k in labels.keys():
    print(str(k) + " : " + str(labels[k]))

## Utilizing Degrees of Freedom
Our very expensive, 64-experiment full factorial design (the data for which maps  (x$_1$,x$_2$,…,x$_6$) to (y$_1$,y$_2$,y$_3$) gives us 64 data points, and 64 degrees of freedom. What we do with those 64 degrees of freedom is up to us.

We could fit an empirical model, or response surface, that has 64 independent parameters, and account for many of the high-order interaction terms - all the way up to six-variable interaction effects. However, high-order effects are rarely important, and are a waste of our degrees of freedom.

Alternatively, we can fit an empirical model with fewer coefficients, using up fewer degrees of freedom, and use the remaining degrees of freedom to characterize the error introduced by our approximate model.

To describe a model with the 3 variables listed above and no other variable interaction effects would use only 3 degrees of freedom, plus 1 degree of freedom for the constant term, leaving 60 degrees of freedom available to quantify error, attribute variance, etc.

Our goal is to use least squares to compute model equations for  (y$_1$,y$_2$,y$_3$)
  as functions of  (x$_1$,x$_4$,x$_6$)
 .

In [ ]:
xlabs = ['x1','x4','x6']
ylabs = ['y1','y2','y3']
ls_data = doe[xlabs+ylabs]

In [ ]:
import statsmodels.api as sm
import numpy as np

x = ls_data[xlabs]
x = sm.add_constant(x)

The first ordinary least squares linear model is created to predict values of the first variable,  y$_1$, as a function of each of our input variables, the list of which are contained in the xlabs variable. When we perform the linear regression fitting, we see much of the same information that we found in the prior two-level three-factor full factorial design, but here, everything is done automatically.

The model is linear, meaning it's fitting the coefficients of the function:
\begin{equation}
\hat{y}=a_0+a_1x_1+a_2x_2+a_3+x_3+a_4x_4+a_5x_5+a_6x_6
\end{equation} 
(here, the variables y and x are vectors, with one component for each response; in our case, they are three-dimensional vectors.)

Because there are 64 observations and 7 coefficients, the 57 extra observations give us extra degrees of freedom with which to assess how good the model is. That analysis can be done with an ordinary least squares (OLS) model, available through the statsmodel library in Python.

## Ordinary Least Squares Regression Model
This built-in OLS model will fit an input vector  (x$_1$,x$_2$,x$_3$,x$_4$,x$_5$,x$_6$)
  to an output vector  (y$_1$,y$_2$,y$_3$)
  using a linear model; the OLS model is designed to fit the model with more observations than coefficients, and utilize the remaining data to quantify the fit of the model.

Let's run through one of these, and analyze the results:

In [ ]:
y1 = ls_data['y1']
est1 = sm.OLS(y1,x).fit()
print(est1.summary())

The StatsModel OLS object prints out quite a bit of useful information, in a nicely-formatted table. Starting at the top, we see a couple of important pieces of information: specifically, the name of the dependent variable (the response) that we're looking at, the number of observations, and the number of degrees of freedom.

We can see an  R$^2$ statistic, which indicates how well this data is fit with our linear model, and an adjusted R$^2$
  statistic, which accounts for the large nubmer of degrees of freedom. While an adjusted R$^2$
  of 0.565 is not great, we have to remember that this linear model is trying to capture a wealth of complexity in six coefficients. Furthermore, the adjusted  R$^2$
  value is too broad to sum up how good our model actually is.

The table in the middle is where the most useful information is located. The coef column shows the coefficients  a$_0$,a$_1$,a$_2$,…
  for the model equation:
\begin{equation}
\hat{y}=a_0+a_1x_1+a_2x_2+a_3+x_3+a_4x_4+a_5x_5+a_6x_6
\end{equation}
 
Using the extra degrees of freedom, an estime s$^2$
  of the variance in the regression coefficients is also computed, and reported in the the std err column. Each linear term is attributed the same amount of variance,  ±0.24
 .

In [ ]:
y2 = ls_data['y2']
est2 = sm.OLS(y2,x).fit()
print(est2.summary())

In [ ]:
y3 = ls_data['y3']
est3 = sm.OLS(y3,x).fit()
print(est3.summary())

## Quantifying Model Goodness-of-Fit
We can now use these linear models to evaluate each set of inputs and compare the model response  $\hat{y}$ to the actual observed response y. What we would expect to see, if our model does an adequate job of representing the underlying behavior of the model, is that in each of the 64 experiments, the difference between the model prediction M and the measured data d, defined as the residual r, r=|d−M|, should be comparable across all experiments. If the residuals appear to have functional dependence on the input variables, it is an indication that our model is missing important effects and needs more or different terms. The way we determine this, mathematically, is by looking at a quantile-quantile plot of our errors (that is, a ranked plot of our error magnitudes).

If the residuals are normally distributed, they will follow a straight line; if the plot shows the data have significant wiggle and do not follow a line, it is an indication that the errors are not normally distributed, and are therefore skewed (indicating terms missing from our OLS model).

In [ ]:
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from matplotlib.pyplot import *

# Quantify goodness of fit

fig = figure(figsize=(14,4))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

r1 = y1 - est1.predict(x)
r2 = y2 - est2.predict(x)
r3 = y3 - est3.predict(x)

stats.probplot(r1, dist="norm", plot=ax1)
ax1.set_title('Residuals, y1')

stats.probplot(r2, dist="norm", plot=ax2)
ax2.set_title('Residuals, y2')

stats.probplot(r3, dist="norm", plot=ax3)
ax3.set_title('Residuals, y3')

Determining whether significant trends are being missed by the model depends on how many points deviate from the red line, and how significantly. If there is a single point that deviates, it does not necessarily indicate a problem; but if there is significant wiggle and most points deviate significantly from the red line, it means that there is something about the relationship between the inputs and the outputs that our model is missing.

There are only a few points deviating from the red line. We saw from the effect quantile for y$_3$
  that there was an interaction variable that was important to modeling the response  y$_3$
 , and it is likely this interaction that is leading to noise at the tail end of these residuals. This indicates residual errors (deviations of the model from data) that do not follow a natural, normal distribution, which indicates there is a pattern in the deviations - namely, the interaction effect.

The conclusion about the error from the quantile plots above is that there are only a few points deviation from the line, and no particularly significant outliers. Our model can use some improvement, but it's a pretty good first-pass model.

## Distribution of Error
Another thing we can look at is the normalized error: what are the residual errors (differences between our model prediction and our data)? How are their values distributed?

A kernel density estimate (KDE) plot, which is a smoothed histogram, shows the probability distribution of the normalized residual errors. As expected, they're bunched pretty close to zero. There are some bumps far from zero, corresponding to the outliers on the quantile-quantile plot of the errors above. However, they're pretty close to randomly distributed, and therefore it doesn't look like there is any systemic bias there.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

fig = figure(figsize=(10,12))
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
axes = [ax1,ax2,ax3]

colors = sns.xkcd_palette(["windows blue", "amber", "faded green", "dusty purple","aqua blue"])

#resids = [r1, r2, r3]
normed_resids = [r1/y1, r2/y2, r3/y3]

for (dataa, axx, colorr) in zip(normed_resids,axes,colors):
    sns.kdeplot(dataa, ax=axx, color=colorr, shade=True, alpha=0.5);

ax1.set_title('Probability Distribution: Normalized Residual Error, y1')
ax2.set_title('Normalized Residual Error, y2')
ax3.set_title('Normalized Residual Error, y3')

Note that in these figures, the bumps at extreme value are caused by the fact that the interval containing the responses includes 0 and values close to 0, so the normalization factor is very tiny, leading to large values.

## Analysis of Variance

In [ ]:
xlabs = ['x1','x4', 'x6']

doe.groupby(xlabs)[ylabs].mean()

In [ ]:
doe.groupby(xlabs)[ylabs].var()

## Distributions of Variance
We can convert these dataframes of averages, variances, and counts into data for plotting. For example, if we want to make a histogram of every value in the groupby dataframe, we can use the .values method, so that this:
´´´
doe.gorupby(xlabs)[ylabs].mean()
´´´
becomes this:

doe.groupby(xlabs)[ylabs].mean().values


This  M×N
  array can then be flattened into a vector using the ravel() method from numpy:

np.ravel( doe.groupby(xlabs)[ylabs].mean().values )

The resulting data can be used to generate histograms, as shown below:

In [ ]:
# Histogram of means of response values, grouped by xlabs

xlabs = ['x1','x4','x6']

print("Grouping responses by %s"%( "-".join(xlabs) ))

dat = np.ravel(doe.groupby(xlabs)[ylabs].mean().values) / np.ravel(doe.groupby(xlabs)[ylabs].var().values)

hist(dat, 10, color=colors[3]);
xlabel(r'Relative Variance ($\mu$/$\sigma^2$)')
show()

In [ ]:
# Histogram of variances of response values, grouped by xlabs

print("Grouping responses by %s"%( "-".join(xlabs) ))

dat = np.ravel(doe.groupby(xlabs)['y1'].var().values)

hist(dat, color=colors[4])
xlabel(r'Variance in $y_{1}$ Response')
ylabel(r'Frequency')
show()

In [ ]:
# Histogram of variances of response values, grouped by xlabs

print("Grouping responses by %s"%( "-".join(xlabs) ))

dat = np.ravel(doe.groupby(xlabs)['y2'].var().values)

hist(dat, color=colors[4])
xlabel(r'Variance in $y_{2}$ Response')
ylabel(r'Frequency')
show()

In [ ]:
# Histogram of variances of response values, grouped by xlabs

print("Grouping responses by %s"%( "-".join(xlabs) ))

dat = np.ravel(doe.groupby(xlabs)['y3'].var().values)

hist(dat, color=colors[4])
xlabel(r'Variance in $y_{3}$ Response')
ylabel(r'Frequency')
show()

This reflects that y1 is best described by the chosen parameters... 

## Residual vs. Response Plots
Another thing we can do, to look for uncaptured effects, is to look at our residuals vs.  $\hat{y}$. This is a further effort to look for underlying functional relationships between  $\hat{y}$
  and the residuals, which would indicate that our system exhibits behavior not captured by our linear model.

In [ ]:
# normal plot of residuals

fig = figure(figsize=(14,4))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

ax1.plot(y1,r1,'o',color=colors[0])
ax1.set_xlabel('Response value $y_1$')
ax1.set_ylabel('Residual $r_1$')

ax2.plot(y2,r2,'o',color=colors[1])
ax2.set_xlabel('Response value $y_2$')
ax2.set_ylabel('Residual $r_2$')
ax2.set_title('Response vs. Residual Plots')

ax3.plot(y1,r1,'o',color=colors[2])
ax3.set_xlabel('Response value $y_3$')
ax3.set_ylabel('Residual $r_3$')

show()

Notice that each plot is trending up and to the right - indicative of an underlying trend that our model  $\hat{y}$
  is not capturing. The trend is relatively weak, however, indicating that our linear model does a good job of capturing most of the relevant effects of this system.

## Discussion
The analysis shows that there are some higher-order or nonlinear effects in the system that a purely linear model does not account for. Next steps would involve adding higher order points for a quadratic or higher order polynomial model to gather additional data to fit the higher-degree models.